In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt

#### #Task 1: Merging 12 months of sales data into a single file

In [ ]:
#df = pd.read_csv("file path"), use forward slashes
df = pd.read_csv("./Pandas_Data_Science_Tasks_master/SalesAnalysis/Sales_Data/")
#reading files in a directory
files = [file for file in os.listdir("./Pandas_Data_Science_Tasks_master/SalesAnalysis/Sales_Data")]

#Empty dataframe creation 
all_months_data = pd.DataFrame()

for file in files:
    df = pd.read_csv("./Pandas_Data_Science_Tasks_master/SalesAnalysis/Sales_Data/" + file)
    all_months_data = pd.concat([all_months_data, df])


all_months_data.head()
all_months_data.to_csv("all_data.csv", index=False)

### Read in updated dataframe

In [ ]:
all_data = pd.read_csv("all_data.csv")
all_data.head()

### Clean up data!

### DROPPING NANs

In [ ]:
#Checking for nan values
df_nan = all_data[all_data.isna().any(axis=1)]
df_nan

#how="any" drops a row with just 1, "all" drops a row with all
all_data = all_data.dropna(how="all")

### Find "Or" and delete

In [ ]:
all_data = all_data[all_data["Order Date"].str[0:2] != "Or"]
all_data

### Augment Data With additional Columns

### Task 2: Adding the month column

In [ ]:
all_data["Month"] = all_data["Order Date"].str[0:2]
all_data["Month"] = all_data["Month"].astype("int32")
all_data.head()

### Task 3: Add a sales column

### Convert to correct column types

In [ ]:
#Convert datatypes to floats to multiply
all_data.dtypes
all_data["Quantity Ordered"] = all_data["Quantity Ordered"].astype("int32")
all_data["Price Each"] = all_data["Price Each"].astype("float")

#ALTERNATIVELY
#all_data["Quantity Ordered"] = pd.tonumeric.(all_data["Quantity Ordered"])
#all_data["Price Each"] = pd.tonumeric(all_data["Price Each"])

In [ ]:
all_data["Sales"] = all_data["Price Each"] * all_data["Quantity Ordered"]
#all_data.columns.get_loc("key")

#REORDERING
#col = all_data.pop('Sales')
#all_data.insert(4, col.name, col)
 

all_data.reset_index(inplace=True)
all_data.drop(columns = ["index"])

#### Task 4: Add a city Column

In [ ]:
#Getting the cities
def get_city(address):
    return address.split(",")[1]

def get_state(address):
    return address.split(",")[2].split(" ")[1]

#Using apply method to access any function
all_data["City"] = all_data["Purchase Address"].apply(lambda x: f"{get_city(x)} ({get_state(x)})")
all_data

# Question 1: Month with the best sales?

In [ ]:
res = all_data.groupby("Month").sum()

In [ ]:
import matplotlib.pyplot as plt

months = range(1, 13)

plt.bar(months, res["Sales"])

plt.xticks(months)
plt.ylabel("Sales in USD ($)")
plt.xlabel("Month number")
plt.show()


### Question 2: What city had the highest number of sales

In [ ]:
sales = all_data.groupby("City").sum()
sales

In [ ]:
#cities = all_data["City"].unique() ---> The x axis orders the unique values according to where they appear, above function resolves this
#Corrects the order of the x axis
cities = [city for city, df in all_data.groupby("City")]

plt.bar(cities, sales["Sales"]) 
plt.xticks(cities, rotation="vertical", size = 7)
plt.ylabel("Sales in USD ($)")
plt.xlabel("City name")
plt.show()

##### Question 3: What Time should we display ads to maximize likelihood of customer's buying product?

In [ ]:
all_data.head()

def get_time(date):
    return date.split(" ")[1]

all_data["Time"] = all_data["Order Date"].apply(lambda a: get_time(a))
all_data["Time"]

In [ ]:
#Converting to date-time format
all_data.drop(columns="index", inplace=True)

all_data["Order Date"] = pd.to_datetime(all_data["Order Date"])
all_data.head()

In [ ]:
#Grabbing the hours 
all_data["Hour"] = all_data["Order Date"].dt.hour
#Grabbing the hours 
all_data["minute"] = all_data["Order Date"].dt.minute
all_data.head()


In [ ]:

hours = [hour for hour, df in all_data.groupby("Hour")]

plt.plot(hours, all_data.groupby(["Hour"]).count())
plt.xticks(hours)
plt.grid()
plt.xlabel("HOURS")
plt.ylabel("Number of Orders")


all_data.groupby(["Hour"]).count()
